# iNaturalist status updates by state

Using the file produced in the collate-status-taxa.ipynb: `inat-aust-status-taxa.csv`, generate lists to update iNaturalist statuses

**Next steps:**
State by state establish the changes that need to be made:
    a. new - any new species that appear in the state lists but do not have a status in inaturalist (new template)
    b. updates - any changes to information which was added by us previously (user_id = 708886) (update template, action='UPDATE')
    c. removals - any statuses which were added by us previously (user_id = 708886) list which are incorrect (update template, action='REMOVE')
    d. flags - are there any statuses by other users that need to be flagged?

## Prep - common to all states
1. Read in the inaturalist statuses & filter out this state
2. Read in the inaturalist taxa list
3. Read in the state sensitive list
4. Attempt to match the state statuses to an IUCN equivalent


### 1. iNaturalist statuses

In [28]:
import pandas as pd

# projectdir = "/Users/oco115/PycharmProjects/authoritative-lists/" # basedir for this gh project
projectdir = "/Users/new330/IdeaProjects/authoritative-lists/" # basedir for this gh project
sourcedir = projectdir + "source-data/inaturalist-statuses/"
listdir = projectdir + "current-lists/"


# read in the statuses
taxastatus = pd.read_csv(sourcedir + "inat-aust-status-taxa.csv", encoding='UTF-8',na_filter=False,dtype=str) ## Read inaturalist conservation statuses file
taxastatus.head(3)

,id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
0,166449,38493,1138587,7830,,Flora and Fauna Guarantee Act 1988,CR,,,obscured,...,Eulamprus,kosciuskoi,,2021-03-01T10:35:01Z,Eulamprus kosciuskoi,species,http://reptile-database.reptarium.cz/search.ph...,,,
1,234788,918383,702203,9994,,Atlas of Living Australia,NT,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,
2,234789,918383,702203,7308,,Atlas of Living Australia,LC,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,


In [30]:
def filter_state_statuses(stateregex: str, urlregex: str):
    authoritydf = taxastatus['authority'].drop_duplicates().sort_values()
    authoritydf = authoritydf[pd.Series(authoritydf).str.contains(stateregex)]
    urldf = taxastatus['url'].drop_duplicates().sort_values()
    urldf = urldf[pd.Series(urldf).str.contains(urlregex)]
    placedisplaydf = taxastatus['place_display_name'].drop_duplicates().sort_values()
    placedisplaydf = placedisplaydf[pd.Series(placedisplaydf).str.contains(stateregex)]
    placedf = taxastatus['place_name'].drop_duplicates().sort_values()
    placedf = placedf[pd.Series(placedf).str.contains(stateregex)]
    # concat all and remove duplicates
    statedf = pd.concat([taxastatus.apply(lambda row: row[taxastatus['place_display_name'].isin(placedisplaydf)]),
                         taxastatus.apply(lambda row: row[taxastatus['place_name'].isin(placedf)]),
                         taxastatus.apply(lambda row: row[taxastatus['url'].isin(urldf)]),
                         taxastatus.apply(
                             lambda row: row[taxastatus['authority'].isin(authoritydf)])]).drop_duplicates()
    return statedf.sort_values(['taxon_id', 'user_id'])


inatstatuses = filter_state_statuses("Qld|QLD|Queensland|QUEENSLAND|QL", ".qld.")
inatstatuses.rename(columns={'id':'status_id','id_y':'taxon_id_y'},inplace=True)
inatstatuses

,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
532,223098,1003032,3669610,7308,,Queensland Nature Conservation Act 1992,CE,https://apps.des.qld.gov.au/species-search/det...,,obscured,...,Antrophyum,austroqueenslandicum,,2021-10-01T10:37:36Z,Antrophyum austroqueenslandicum,species,,,,
131,169944,1004434,58320,7308,,Queensland Nature Conservation Act 1992,Least concern,https://apps.des.qld.gov.au/species-search/det...,Listed as 'confidential' by the Queensland Dep...,obscured,...,Oberonia,crateriformis,,2021-07-18T09:05:24Z,Oberonia crateriformis,species,http://www.catalogueoflife.org/annual-checklis...,,,
1473,152757,100615,708886,7308,16653,QLD DEHP,endangered,https://data.qld.gov.au/dataset/conservation-s...,,obscured,...,Euastacus,bindal,,2020-05-28T05:44:48Z,Euastacus bindal,species,http://www.iucnredlist.org/apps/redlist/details,,,
1407,152668,100635,708886,7308,16653,QLD DEHP,endangered,https://data.qld.gov.au/dataset/conservation-s...,,obscured,...,Euastacus,jagara,,2020-05-28T05:43:40Z,Euastacus jagara,species,http://www.iucnredlist.org/apps/redlist/details,,,
1267,152461,100638,708886,7308,16653,QLD DEHP,endangered,https://data.qld.gov.au/dataset/conservation-s...,,obscured,...,Euastacus,maidae,,2020-05-28T05:06:01Z,Euastacus maidae,species,http://www.iucnredlist.org/apps/redlist/details,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,262047,977629,3669610,7308,,Queensland Nature Conservation Act 1992,NT,https://www.data.qld.gov.au/dataset/conservati...,,open,...,Melaleuca,formosa,,2022-09-05T10:47:52Z,Melaleuca formosa,species,,,,
2615,161807,993333,702203,7308,,Nature Conservation Act 1992,VU,https://apps.des.qld.gov.au/species-search/det...,,,...,Cupaniopsis,tomentella,,2020-08-30T17:12:00Z,Cupaniopsis tomentella,species,https://eol.org/pages/5629346,,,
2616,161808,993333,702203,6744,,Environment Protection and Biodiversity Conser...,VU,https://apps.des.qld.gov.au/species-search/det...,,,...,Cupaniopsis,tomentella,,2020-08-30T17:12:00Z,Cupaniopsis tomentella,species,https://eol.org/pages/5629346,,,
23,167723,993605,3669610,7308,,QLD DEHP,NT,https://www.data.qld.gov.au/dataset/conservati...,,obscured,...,Acianthus,amplexicaulis,,2021-10-05T08:48:02Z,Acianthus amplexicaulis,species,http://www.catalogueoflife.org/annual-checklis...,,,


### 2. iNaturalist taxonomy

In [3]:
# Output files contain these fields
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# so we need to match species from the state lists to the inat taxa to get the taxon_id

import zipfile
url = "https://www.inaturalist.org/taxa/inaturalist-taxonomy.dwca.zip"
filename = url.split("/")[-1]

z=zipfile.ZipFile(sourcedir + filename)

with z.open('taxa.csv') as from_archive:
    inattaxa = pd.read_csv(from_archive,dtype=str)
z.close()
inattaxa.head(3)


,id,taxonID,identifier,parentNameUsageID,kingdom,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references
0,1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/48460,Animalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-02T06:05:44Z,Animalia,kingdom,http://www.catalogueoflife.org/annual-checklis...
1,2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/1,Animalia,Chordata,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-23T00:40:18Z,Chordata,phylum,http://www.catalogueoflife.org/annual-checklis...
2,3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/355675,Animalia,Chordata,Aves,NaN,NaN,NaN,NaN,NaN,2022-12-27T07:33:16Z,Aves,class,http://www.catalogueoflife.org/annual-checklis...


### 3. State lists

In [4]:
sensitivelist = pd.read_csv(listdir + "sensitive-lists/QLD-sensitive.csv")  # Qld sensitive list
sensitivelist['scientificName'] = sensitivelist['scientificName'].str.replace('subsp. ', '', regex=False)
sensitivelist = sensitivelist.rename(columns={'taxonID':'wildnetTaxonID'})
sensitivelist

,wildnetTaxonID,kingdom,class,family,scientificName,vernacularName,scientificNameAuthorship,sourceStatus,Significant,status,Endemicity,EPBC Status
0,969,Animalia,Mammalia,Rhinolophidae,Rhinolophus philippinensis,greater large-eared horseshoe bat,"Waterhouse, 1843",E,Y,Endangered,Regional Endemic,Vulnerable
1,1376,Animalia,Aves,Estrildidae,Chloebia gouldiae,Gouldian finch,"(Gould, 1844)",E,Y,Endangered,Intranational,Endangered
2,1378,Animalia,Aves,Estrildidae,Erythrura trichroa,blue-faced parrot-finch,"(Kittlitz, 1835)",NT,Y,Near Threatened,Not Endemic to Australia,NaN
3,1370,Animalia,Aves,Estrildidae,Neochmia phaeton evangelinae,crimson finch (white-bellied subspecies),"(Hombron & Jacquinot, 1841)",E,Y,Endangered,Regional Endemic,Endangered
4,1365,Animalia,Aves,Estrildidae,Poephila cincta cincta,black-throated finch (white-rumped subspecies),"Gould, 1837",E,Y,Endangered,Intranational,Endangered
...,...,...,...,...,...,...,...,...,...,...,...,...
950,11699,Plantae,Equisetopsida,Thelypteridaceae,Pneumatopteris costata,NaN,(Brack.) Holttum,NT,Y,Near Threatened,Regional Endemic,NaN
951,11700,Plantae,Equisetopsida,Thelypteridaceae,Pneumatopteris pennigera,lime fern,(G.Forst.) Holttum,E,Y,Endangered,Not Endemic to Australia,NaN
952,16042,Plantae,Equisetopsida,Thelypteridaceae,Thelypteris confluens,NaN,(Thunb.) C.V.Morton,V,Y,Vulnerable,Not Endemic to Australia,NaN
953,8185,Plantae,Equisetopsida,Proteaceae,Macadamia jansenii,NaN,C.L.Gross & P.H.Weston,CR,Y,Critically Endangered,Queensland Endemic,Endangered


### 4. Equivalent IUCN statuses

In [5]:
iucn_statuses = {'Not Evaluated', 'Data Deficient', 'Least Concern', 'Near Threatened', 'Vulnerable', 'Endangered', 'Critically Endangered', 'Extinct in the Wild' and 'Extinct'}
sensitivelist.groupby(['status'])['status'].count()

status
Critically Endangered     35
Endangered                78
Extinct in the wild        7
Least concern            143
Near Threatened           63
Special least concern    361
Vulnerable               139
Name: status, dtype: int64

In [6]:
iucnStatusMappings = {
    'critically endangered': 'Critically Endangered',
    'vulnerable':'Vulnerable',
    'not evaluated':'Not Evaluated',
    'data deficient':'Data Deficient',
    'least concern':'Least Concern',
    'special least concern':'Least Concern',
    'near threatened':'Near Threatened',
    'endangered':'Endangered',
    'extinct in the wild':'Extinct in the Wild',
    'extinct':'Extinct',
    'confidential':'Vulnerable'
}

### 5. Determine best place ID to use

In [7]:
inatstatuses.groupby(['place_id','place_name','place_display_name'])['place_id'].count()
# looks like 7308

place_id  place_name             place_display_name           
                                                                    5
144315    Brisbane City          Brisbane City                      2
153119    South East Queensland  South East Queensland, QL, AU      1
18870     Cairns - Pt B          Cairns - Pt B, QL, AU              1
19232     Yarrabah               Yarrabah, QL, AU                   1
6744      Australia              Australia                         12
7308      Queensland             Queensland, AU                   631
Name: place_id, dtype: int64

## Merge iNaturalist statuses with State sensitive list on scientificName

1. Match - updates, even if the statuses are the same we'll update the links and values anyway
2. No match - statuses to be added (additions)
   1.1 No match and no taxnomy - search for synonyms
   1.2 No match
3. Merge the other direction to see if there are deletes?


In [54]:
# join to see which lists already have a status in inaturalist based on scientificName
mergedstatuses = sensitivelist[['wildnetTaxonID','scientificName','status']].merge(inatstatuses[['status_id','scientificName','taxon_id','user_id','description','iucn','authority','status','geoprivacy','place_id','place_display_name']],how="left",left_on='scientificName',right_on='scientificName',suffixes=(None,'_inat')).sort_values(['scientificName'])
mergedstatuses


,wildnetTaxonID,scientificName,status,status_id,taxon_id,user_id,description,iucn,authority,status_inat,geoprivacy,place_id,place_display_name
127,9655,Acianthus,Least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,7974,Acianthus borealis,Special least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,13477,Acianthus caudatus,Special least concern,152636,320644,708886,,40,QLD DEHP,endangered,obscured,7308,"Queensland, AU"
130,14086,Acianthus exsertus,Special least concern,152637,323899,708886,,10,QLD DEHP,LC,,7308,"Queensland, AU"
131,14087,Acianthus fornicatus,Special least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,8539,Wodyetia bifurcata,Vulnerable,152450,135348,708886,,40,QLD DEHP,endangered,obscured,7308,"Queensland, AU"
55,22659,Wollumbinia belli,Vulnerable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
778,21765,Zeuxine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
779,40091,Zeuxine attenuata,Least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# prepare the export fields, common to New template and Update template
# new statuses
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# updates
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username
mergedstatuses['new_authority'] = "Qld Department of Environment and Science"
mergedstatuses['new_description'] = "Listed as Confidential - refer to https://www.data.qld.gov.au/dataset/queensland-confidential-species"
mergedstatuses['new_url'] = "https://apps.des.qld.gov.au/species-search/details/?id=" + mergedstatuses['wildnetTaxonID'].astype(str)
mergedstatuses['new_geoprivacy'] = "obscured"
mergedstatuses['new_place_id'] = '7308'  # Queensland, AU
mergedstatuses['new_username'] = 'peggydnew'
mergedstatuses['new_iucn_equivalent'] = mergedstatuses['status'].str.lower().str.strip().map(iucnStatusMappings).fillna('Vulnerable') # map to dictionary
mergedstatuses['new_status'] = mergedstatuses['status'].fillna('Confidential')
mergedstatuses

,wildnetTaxonID,scientificName,status,status_id,taxon_id,user_id,description,iucn,authority,status_inat,...,place_id,place_display_name,new_authority,new_description,new_url,new_geoprivacy,new_place_id,new_username,new_iucn_equivalent,new_status
127,9655,Acianthus,Least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Least Concern,Least concern
128,7974,Acianthus borealis,Special least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Least Concern,Special least concern
129,13477,Acianthus caudatus,Special least concern,152636,320644,708886,,40,QLD DEHP,endangered,...,7308,"Queensland, AU",Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Least Concern,Special least concern
130,14086,Acianthus exsertus,Special least concern,152637,323899,708886,,10,QLD DEHP,LC,...,7308,"Queensland, AU",Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Least Concern,Special least concern
131,14087,Acianthus fornicatus,Special least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Least Concern,Special least concern
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,8539,Wodyetia bifurcata,Vulnerable,152450,135348,708886,,40,QLD DEHP,endangered,...,7308,"Queensland, AU",Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Vulnerable,Vulnerable
55,22659,Wollumbinia belli,Vulnerable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Vulnerable,Vulnerable
778,21765,Zeuxine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Vulnerable,Confidential
779,40091,Zeuxine attenuata,Least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Qld Department of Environment and Science,Listed as Confidential - refer to https://www....,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Least Concern,Least concern


## Updates

In [73]:
# updates
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
updates = pd.DataFrame(mergedstatuses[mergedstatuses['status_id'].notnull()])
updates.sort_values('scientificName')
updates['action'] = 'UPDATE'
#updates.loc[:,'action'] = 'UPDATE'
updates = updates[['action','scientificName','status_id','taxon_id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
updates.columns = updates.columns.str.replace("new_", "", regex=True)
updates = updates.rename(columns={'scientificName':'taxon_name',
                                  'status_id':'id'})
updates

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
129,UPDATE,Acianthus caudatus,152636,320644,Special least concern,Least Concern,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
130,UPDATE,Acianthus exsertus,152637,323899,Special least concern,Least Concern,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
133,UPDATE,Acianthus pusillus,152632,549968,Special least concern,Least Concern,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
134,UPDATE,Acianthus saxatilis,152567,739107,Endangered,Endangered,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
135,UPDATE,Acriopsis,152867,425476,Least concern,Least Concern,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
...,...,...,...,...,...,...,...,...,...,...,...,...
773,UPDATE,Vanda hindsii,152654,140333,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
39,UPDATE,Varanus semiremex,152553,39474,Least concern,Least Concern,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
776,UPDATE,Vrydagzynea,152846,426364,Confidential,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
126,UPDATE,Wodyetia bifurcata,152450,135348,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....


In [59]:
# investigation - which updates are mine (346), which are those not from me (80 not from me)
#updates[updates['user_id']=='708886'][['scientificName','new_status','status_inat','authority','new_authority','description','new_description','geoprivacy','new_geoprivacy']]
#updates[updates['user_id']!='708886'][['user_id','scientificName','new_status','status_inat','authority','new_authority','description','new_description','geoprivacy','new_geoprivacy']]
# those with different statuses
#updates[updates['new_status'].str.lower().str.strip() != updates['status_inat'].str.lower().str.strip()][['scientificName','new_status','status_inat','authority','new_authority','description','new_description','geoprivacy','new_geoprivacy']]
# users who've updated qld statuses who aren't me
#'https://www.inaturalist.org/users/220795','Steven Kurniawidjaja','neontetraploid','US'
#'https://www.inaturalist.org/users/3669610','Craig Robbins','craig-r','AU'
#'https://www.inaturalist.org/users/527710','James Kameron Mitchell','jameskm','US'
#'https://www.inaturalist.org/users/58320','lwnrngr','lwnrngr','NZ'
#'https://www.inaturalist.org/users/702203','Kitty Maurey','kitty12','CA'
#'https://www.inaturalist.org/users/717122','Miguel de Salas','mftasp','TAS'


,scientificName,new_status,status_inat,authority,new_authority,description,new_description,geoprivacy,new_geoprivacy
129,Acianthus caudatus,Special least concern,endangered,QLD DEHP,Qld Department of Environment and Science,,Listed as Confidential - refer to https://www....,obscured,obscured
130,Acianthus exsertus,Special least concern,LC,QLD DEHP,Qld Department of Environment and Science,,Listed as Confidential - refer to https://www....,,obscured
133,Acianthus pusillus,Special least concern,LC,QLD DEHP,Qld Department of Environment and Science,Least concern (Nature Conservation Act 1992) b...,Listed as Confidential - refer to https://www....,obscured,obscured
134,Acianthus saxatilis,Endangered,endangered,QLD DEHP,Qld Department of Environment and Science,,Listed as Confidential - refer to https://www....,obscured,obscured
135,Acriopsis,Least concern,Least concern,Queensland Nature Conservation Act 1992,Qld Department of Environment and Science,Listed as 'confidential' by the Queensland Dep...,Listed as Confidential - refer to https://www....,obscured,obscured
...,...,...,...,...,...,...,...,...,...
773,Vanda hindsii,Vulnerable,Vulnerable,Queensland Nature Conservation Act 1992,Qld Department of Environment and Science,,Listed as Confidential - refer to https://www....,obscured,obscured
39,Varanus semiremex,Least concern,endangered,QLD DEHP,Qld Department of Environment and Science,,Listed as Confidential - refer to https://www....,obscured,obscured
776,Vrydagzynea,Confidential,Confidential,Queensland Government,Qld Department of Environment and Science,Listed as 'confidential' by the Queensland Dep...,Listed as Confidential - refer to https://www....,obscured,obscured
126,Wodyetia bifurcata,Vulnerable,endangered,QLD DEHP,Qld Department of Environment and Science,,Listed as Confidential - refer to https://www....,obscured,obscured


## No status in iNaturalist via straight scientificName match
The Qld records that didn't match up to a status in iNaturalist

In [81]:
# to add: those that have no inaturalist status - 532!!
noinatstatus = mergedstatuses[mergedstatuses['status_id'].isnull()]
# try to match the taxon name to something in inaturalist
noinatstatus = noinatstatus.merge(inattaxa, how="left", left_on="scientificName",right_on="scientificName")
noinatstatus

,wildnetTaxonID,scientificName,status,status_id,taxon_id,user_id,description,iucn,authority,status_inat,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
0,9655,Acianthus,Least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Acianthus,NaN,NaN,2021-04-17T03:13:28Z,genus,http://www.catalogueoflife.org/annual-checklis...
1,7974,Acianthus borealis,Special least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14087,Acianthus fornicatus,Special least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Acianthus,fornicatus,NaN,2021-05-20T01:27:37Z,species,http://www.catalogueoflife.org/annual-checklis...
3,8545,Acianthus ledwardii,Special least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Acianthus,ledwardii,NaN,2021-05-18T04:43:56Z,species,http://www.catalogueoflife.org/annual-checklis...
4,31909,Acriopsis emarginata,Vulnerable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Acriopsis,emarginata,NaN,2020-11-16T22:24:21Z,species,http://www.catalogueoflife.org/annual-checklis...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,31261,Vanilla planifolia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Vanilla,planifolia,NaN,2019-11-23T05:28:46Z,species,http://www.catalogueoflife.org/annual-checklis...
533,30794,Vrydagzynea grayi,Endangered,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
534,22659,Wollumbinia belli,Vulnerable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
535,21765,Zeuxine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Zeuxine,NaN,NaN,2022-10-17T14:37:53Z,genus,http://www.ubio.org/browser/details.php?nameba...


In [91]:
additions = pd.DataFrame(noinatstatus[noinatstatus['id'].notna()])
additions
# note: "Dendrobium" matches to both genus and section

,wildnetTaxonID,scientificName,status,status_id,taxon_id,user_id,description,iucn,authority,status_inat,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
0,9655,Acianthus,Least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Acianthus,NaN,NaN,2021-04-17T03:13:28Z,genus,http://www.catalogueoflife.org/annual-checklis...
2,14087,Acianthus fornicatus,Special least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Acianthus,fornicatus,NaN,2021-05-20T01:27:37Z,species,http://www.catalogueoflife.org/annual-checklis...
3,8545,Acianthus ledwardii,Special least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Acianthus,ledwardii,NaN,2021-05-18T04:43:56Z,species,http://www.catalogueoflife.org/annual-checklis...
4,31909,Acriopsis emarginata,Vulnerable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Acriopsis,emarginata,NaN,2020-11-16T22:24:21Z,species,http://www.catalogueoflife.org/annual-checklis...
6,41744,Alsophila cunninghamii,Near Threatened,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Polypodiopsida,Cyatheales,Cyatheaceae,Alsophila,cunninghamii,NaN,2019-02-17T04:31:42Z,species,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,21627,Thrixspermum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Thrixspermum,NaN,NaN,2022-09-27T21:37:53Z,genus,http://www.catalogueoflife.org/annual-checklis...
525,21895,Trachoma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Trachoma,NaN,NaN,2020-08-09T23:54:56Z,genus,NaN
530,1093,Turnix olivii,Critically Endangered,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Aves,Charadriiformes,Turnicidae,Turnix,olivii,NaN,2019-11-23T00:55:48Z,species,http://www.birdlife.org/datazone/speciesfactsh...
532,31261,Vanilla planifolia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Orchidaceae,Vanilla,planifolia,NaN,2019-11-23T05:28:46Z,species,http://www.catalogueoflife.org/annual-checklis...


In [98]:
# there's no status but there is a matching inat taxon (id is the taxon id)
additions = pd.DataFrame(noinatstatus[noinatstatus['id'].notna()])
additions.sort_values(['scientificName'])
additions['action'] = 'ADD'
additions = additions[['action','scientificName','status_id','id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
additions.columns = additions.columns.str.replace("new_", "", regex=True)
additions = additions.rename(columns={'scientificName':'taxon_name',
                                      'id':'taxon_id',
                                  'status_id':'id'})
additions

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
0,ADD,Acianthus,NaN,202580,Least concern,Least Concern,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
2,ADD,Acianthus fornicatus,NaN,202579,Special least concern,Least Concern,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
3,ADD,Acianthus ledwardii,NaN,1239753,Special least concern,Least Concern,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
4,ADD,Acriopsis emarginata,NaN,1141144,Vulnerable,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
6,ADD,Alsophila cunninghamii,NaN,407141,Near Threatened,Near Threatened,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
...,...,...,...,...,...,...,...,...,...,...,...,...
523,ADD,Thrixspermum,NaN,140357,Confidential,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
525,ADD,Trachoma,NaN,1069342,Confidential,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
530,ADD,Turnix olivii,NaN,20902,Critically Endangered,Critically Endangered,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....
532,ADD,Vanilla planifolia,NaN,61393,Confidential,Vulnerable,Qld Department of Environment and Science,https://apps.des.qld.gov.au/species-search/det...,obscured,7308,peggydnew,Listed as Confidential - refer to https://www....


In [99]:
# write these to the file
pd.concat([updates,additions]).to_csv(sourcedir + "qld.csv", index=False)

In [50]:
# what didnt match to a taxon?
unknownToInat = noinatstatus[additions['id'].isna()]
unknownToInat

,wildnetTaxonID,scientificName,status,status_id,taxon_id,user_id,authority,status_inat,geoprivacy,place_id,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
1,7974,Acianthus borealis,Special least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,40762,Alsophila baileyana,Least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,41582,Alsophila exilis,Endangered,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,30917,Aponogeton cuneatus,Special least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,21902,Aponogeton elongatus elongatus,Near Threatened,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,11357,Trichoglottis australiensis,Vulnerable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
531,40770,Vanda whiteana,Least concern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
533,30794,Vrydagzynea grayi,Endangered,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
534,22659,Wollumbinia belli,Vulnerable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### are there any that need to be removed?
qld sensitive list count: 955
qld inat statuses count: 653

updates to inat status: 426
additional inat status: 278
qld statuses we can't find a taxon match for in iNaturalist: 259
total: 963 (explainable via the various genus/section entries that we matched to in the taxonomy)

inat statuses left over: 653-426=227 that may need checking

In [95]:
# inat statuses that aren't in added or updated
inatstatuses[~inatstatuses['taxon_id'].isin(updates['taxon_id'])]


,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
408,223608,1015555,3669610,7308,,Queensland Nature Conservation Act 1992,NT,https://apps.des.qld.gov.au/species-search/det...,,,...,Symplocos,harroldii,,2022-11-30T22:01:59Z,Symplocos harroldii,species,https://eol.org/pages/47146631,,,
102,159922,1019990,702203,7308,,Queensland,VU,https://www.legislation.qld.gov.au/view/html/i...,,,...,Acacia,baueri,,2022-04-06T22:03:46Z,Acacia baueri,species,http://www.catalogueoflife.org/annual-checklis...,,,
667,165360,1023152,58320,7308,,Queensland Government,Near threatened,https://apps.des.qld.gov.au/species-search/det...,,,...,Bertya,pedicellata,,2021-01-05T06:59:04Z,Bertya pedicellata,species,http://www.catalogueoflife.org/annual-checklis...,,,
3327,161810,1032816,425992,7308,,Queensland Government,VU,https://apps.des.qld.gov.au/species-search/det...,,open,...,Macadamia,ternifolia,,2020-08-30T17:12:56Z,Macadamia ternifolia,species,,,,
263,180953,1033327,702203,7308,,Nature Conservation Act 1992,EN,https://bie.ala.org.au/species/https://id.biod...,,,...,Grevillea,linsmithii,,2021-09-20T02:01:57Z,Grevillea linsmithii,species,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,262047,977629,3669610,7308,,Queensland Nature Conservation Act 1992,NT,https://www.data.qld.gov.au/dataset/conservati...,,open,...,Melaleuca,formosa,,2022-09-05T10:47:52Z,Melaleuca formosa,species,,,,
2615,161807,993333,702203,7308,,Nature Conservation Act 1992,VU,https://apps.des.qld.gov.au/species-search/det...,,,...,Cupaniopsis,tomentella,,2020-08-30T17:12:00Z,Cupaniopsis tomentella,species,https://eol.org/pages/5629346,,,
2616,161808,993333,702203,6744,,Environment Protection and Biodiversity Conser...,VU,https://apps.des.qld.gov.au/species-search/det...,,,...,Cupaniopsis,tomentella,,2020-08-30T17:12:00Z,Cupaniopsis tomentella,species,https://eol.org/pages/5629346,,,
23,167723,993605,3669610,7308,,QLD DEHP,NT,https://www.data.qld.gov.au/dataset/conservati...,,obscured,...,Acianthus,amplexicaulis,,2021-10-05T08:48:02Z,Acianthus amplexicaulis,species,http://www.catalogueoflife.org/annual-checklis...,,,
